![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [70]:
import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso
from sklearn.model_selection import RandomizedSearchCV

In [71]:
# Read the CSV file into a DataFrame
df = pd.read_csv('rental_info.csv')

# Display the first few rows of the DataFrame
df.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [72]:
# Convert the 'return_date' column to datetime format
df['return_date'] = pd.to_datetime(df['return_date'])

# Convert the 'rental_date' column to datetime format
df['rental_date'] = pd.to_datetime(df['rental_date'])

# Calculate the rental length in days by subtracting 'rental_date' from 'return_date'
df['rental_length_days'] = (df['return_date'] - df['rental_date']).dt.days

# Display the 'rental_length_days' column
df['rental_length_days']

0        3
1        2
2        7
3        2
4        4
        ..
15856    6
15857    4
15858    9
15859    8
15860    6
Name: rental_length_days, Length: 15861, dtype: int64

In [73]:
# Add a new column 'deleted_scenes' to the dataframe
# Apply a lambda function to check if 'Deleted Scenes' is present in the 'special_features' column
# If 'Deleted Scenes' is present, assign 1 to the 'deleted_scenes' column, otherwise assign 0
df['deleted_scenes'] = df['special_features'].apply(lambda x: 1 if 'Deleted Scenes' in x else 0)

# Add a new column 'behind_the_scenes' to the dataframe
# Apply a lambda function to check if 'Behind the Scenes' is present in the 'special_features' column
# If 'Behind the Scenes' is present, assign 1 to the 'behind_the_scenes' column, otherwise assign 0
df['behind_the_scenes'] = df['special_features'].apply(lambda x: 1 if 'Behind the Scenes' in x else 0)

In [74]:
# Display information about the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   rental_date         15861 non-null  datetime64[ns, UTC]
 1   return_date         15861 non-null  datetime64[ns, UTC]
 2   amount              15861 non-null  float64            
 3   release_year        15861 non-null  float64            
 4   rental_rate         15861 non-null  float64            
 5   length              15861 non-null  float64            
 6   replacement_cost    15861 non-null  float64            
 7   special_features    15861 non-null  object             
 8   NC-17               15861 non-null  int64              
 9   PG                  15861 non-null  int64              
 10  PG-13               15861 non-null  int64              
 11  R                   15861 non-null  int64              
 12  amount_2            15861 non-nu

In [75]:
# We avoid columns that leak data (e.g., "return_date" and "rental date")
X = df[['amount', 'release_year', 'rental_rate', 'length', 'replacement_cost', 'NC-17', 'PG', 'PG-13', 'R', 'amount_2', 'length_2', 'rental_rate_2', 'deleted_scenes', 'behind_the_scenes']]

# Create the target variable 'y'
y = df['rental_length_days']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=9)

In [76]:
# # Instantiate the Lasso model with a specific alpha
# lasso_model = Lasso(alpha=0.01, random_state=9)

# # Fit the Lasso model to the training data
# lasso_model.fit(X_train, y_train)

# # Get the coefficients of the features (feature importances)
# lasso_coef = lasso_model.coef_

# # Create a mask to select columns with non-zero coefficients
# selected_columns_mask = lasso_coef > 0

# # Subset the training and test features using iloc with the mask
# X_train_selected = X_train.iloc[:, selected_columns_mask]
# X_test_selected = X_test.iloc[:, selected_columns_mask]

# # Print the selected feature names
# selected_feature_names = X.columns[selected_columns_mask]
# print("Selected Features:", selected_feature_names)

In [77]:
# # We avoid columns that leak data (e.g., "return_date" and "rental date")
# X = df[['amount', 'release_year', 'length', 'NC-17', 'PG-13',
#        'behind_the_scenes']]

# # Create the target variable 'y'
# y = df['rental_length_days']

# # Split the data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=9)

In [78]:
print(X.shape, y.shape)

(15861, 14) (15861,)


In [79]:
# Define hyperparameter grids for each model
param_grid_linear_reg = {
    'fit_intercept': [True, False],
    'normalize': [True, False],
}

param_grid_decision_tree_reg = {
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

param_grid_random_forest_reg = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt'],
}

In [80]:
# Create models
linear_reg = LinearRegression()
decision_tree_reg = DecisionTreeRegressor(random_state=9)
random_forest_reg = RandomForestRegressor(random_state=9)

# Initialize RandomizedSearchCV for each model
random_search_linear_reg = RandomizedSearchCV(estimator=linear_reg, param_distributions=param_grid_linear_reg, n_iter=10, scoring='neg_mean_squared_error', cv=5, random_state=9)

random_search_decision_tree_reg = RandomizedSearchCV(estimator=decision_tree_reg, param_distributions=param_grid_decision_tree_reg, n_iter=10, scoring='neg_mean_squared_error', cv=5, random_state=9)

random_search_random_forest_reg = RandomizedSearchCV(estimator=random_forest_reg, param_distributions=param_grid_random_forest_reg, n_iter=10, scoring='neg_mean_squared_error', cv=5, random_state=9)

In [81]:
# Fit the RandomizedSearchCV to the training data for each model
random_search_linear_reg.fit(X_train_selected, y_train)
random_search_decision_tree_reg.fit(X_train_selected, y_train)
random_search_random_forest_reg.fit(X_train_selected, y_train)

# Get the best models with tuned hyperparameters
best_linear_reg = random_search_linear_reg.best_estimator_
best_decision_tree_reg = random_search_decision_tree_reg.best_estimator_
best_random_forest_reg = random_search_random_forest_reg.best_estimator_

In [82]:
# Make predictions with the best models
best_linear_reg_pred = best_linear_reg.predict(X_test_selected)
best_decision_tree_pred = best_decision_tree_reg.predict(X_test_selected)
best_random_forest_pred = best_random_forest_reg.predict(X_test_selected)

In [83]:
# Calculate the MSE of the best models
best_linear_reg_mse = mean_squared_error(y_test, best_linear_reg_pred)
best_decision_tree_mse = mean_squared_error(y_test, best_decision_tree_pred)
best_random_forest_mse = mean_squared_error(y_test, best_random_forest_pred)

print(best_linear_reg_mse)
print(best_decision_tree_mse)
print(best_random_forest_mse)

4.846709676954228
2.6487328677201525
2.4103455094852526


In [84]:
# Our best model
best_model = random_search_random_forest_reg.best_estimator_

# Make predictions on the test set using the best model
best_model_pred = best_model.predict(X_test_selected)

# Calculate the MSE on the test set
best_mse = mean_squared_error(y_test, best_model_pred)

# Save the best model using joblib (you can use pickle as an alternative)
model_filename = "best_model.pkl"
joblib.dump(best_model, model_filename)

# Print the MSE and confirm the model has been saved
print("Best Model MSE on Test Set:", best_mse)
print("Best Model saved as:", model_filename)

Best Model MSE on Test Set: 2.4103455094852526
Best Model saved as: best_model.pkl
